In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
%load_ext autoreload
%autoreload 2

В этой части занятия мы пройдем путь от простой идеи "сделаю свой ChatGPT" до понимания реальных вызовов и решений при развертывании LLM в продакшене

**Что мы изучим:**
- 🔥 Почему простого FastAPI + Transformers недостаточно
- ⚡ Ключевые метрики производительности LLM
- 🏗️ Архитектурные паттерны инференса
- 🚄 Сравнение vLLM и TensorRT-LLM
- 💰 Экономика self-hosting vs API

In [3]:
import os
import sys
import json
import torch
import fastapi
import uvicorn
import transformers
import tensorflow
import numpy
import pandas
import accelerate
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [4]:
# версия питона
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [5]:
# список важных библиотек
print('torch', torch.__version__)
print('transformers', transformers.__version__)
print('tensorflow', tensorflow.__version__)
print('numpy', numpy.__version__)
print('pandas', pandas.__version__)
print('accelerate', accelerate.__version__)
print('fastapi', fastapi.__version__)
print('uvicorn', uvicorn.__version__)

torch 2.6.0+cu124
transformers 4.54.0
tensorflow 2.18.0
numpy 2.0.2
pandas 2.2.2
accelerate 1.9.0
fastapi 0.116.1
uvicorn 0.35.0


In [6]:
# информация о GPU для этого урока
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Found {num_gpus} GPU(s).")

    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
  print("CUDA is not available!")

Found 1 GPU(s).
GPU 0: Tesla T4


In [7]:
!pip install -U huggingface_hub

<div style="text-align: center;">
  <img src="docs/meme.jpg" alt="alt text" style="width:400px;">
</div>

# Часть 1
Здесь мы разберем следующие темы
- 🔥 Почему простого FastAPI + Transformers недостаточно
- ⚡ Ключевые метрики производительности LLM
## 📦 Загрузка модели

**Размеры современных моделей:**
- Llama-3.2-1B: ~5 GB
- Llama-3.1-8B: ~16 GB
- Llama-3.1-70B: ~140 GB
- GPT-4 уровень: 500GB+ (по оценкам)

**Проблемы при загрузке:**
1. **Время загрузки**: Даже 1B модель загружается минуты
2. **Требования к RAM**: Модель должна поместиться в GPU памяти
3. **Версионирование**: Разные версии transformers могут быть несовместимы
4. **Токенизатор**: Отдельно нужно загружать и правильно настраивать

### 🔧 Что нужно для Llama-3.2-1B:
- **GPU память**: минимум 8GB (лучше 16GB)
- **Время загрузки**: 2-5 минут (зависит от диска)
- **Интернет**: Только для первого скачивания
- **Права доступа**: Нужен HuggingFace токен для некоторых моделей


#### Команда для загрузки модели
Подробнее [в карточке модели](https://huggingface.co/microsoft/Phi-3.5-mini-instruct/tree/main)
```python
!huggingface-cli download "microsoft/phi-1_5" --local-dir "./models/phi-1_5" --local-dir-use-symlinks False
```

__Важно__: убедитесь, что вы получили доступ к репозиторию модели, а так же сделали login через токен на вашем компьютере командой
```bash
huggingface-cli login
```

In [8]:
!huggingface-cli download "microsoft/phi-1_5" --local-dir "./models/phi-1_5" --local-dir-use-symlinks False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:141: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 15 files:   0% 0/15 [00:00<?, ?it/s]Downloading 'LICENSE' to 'models/phi-1_5/.cache/huggingface/download/DhCjcNQuMpl4FL346qr3tvNUCgY=.8ab7b4964d147858b89544e8ca33203237161edc.incomplete'

README.md: 0.00B [00:00, ?B/s]

CODE_OF_CONDUCT.md: 100% 444/444 [00:00<00:00, 2.76MB/s]
Download complete. Moving file to models/phi-1_5/CODE_OF_CONDUCT.md



config.json: 100% 736/736 [00:00<00:00, 5.81MB/s]
Download complete. Moving file to models/phi-1_5/config.json
README.md: 7.85kB [00:00, 778kB/s]
Download complete. Moving file to models/phi-1_5/README.md



added_tokens.json: 1.08kB [00:00, 5.02MB/s]
Download complete. Moving file to models/phi-1_5/added_tokens.json


LICENSE: 1.08kB [00:00, 5.0

In [9]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN=userdata.get('hf_token')

if HF_TOKEN:
    login(HF_TOKEN)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token is not set. Please save the token first.")

Successfully logged in to Hugging Face!


In [10]:
model_id = "./models/phi-1_5"  # Наша модель лежит по этому пути
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)

In [11]:
# У некоторых моделей (особенно fine-tuned для инструкций/чатов) нет токена для паддинга (`pad_token`)
# Паддинг нужен для выравнивания длины последовательностей при пакетной обработке (batch processing)
# Без этого токена возникнет ошибка

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [12]:
%%time
prompt = "'What is the day today? "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(f"inputs = \n {inputs}")
input_token_count = inputs['input_ids'].shape[1]
print(f"input_token_count = \n {input_token_count}")
output_sequences = model.generate(
    **inputs,
    max_new_tokens=256,
    num_return_sequences=1,
    temperature=0.1,
    do_sample=True
)

total_token_count = output_sequences.shape[1]
output_token_count = total_token_count - input_token_count

generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(f"tokenizer = n\ {tokenizer}")
print(f"model = n\ {model}")

inputs = 
 {'input_ids': tensor([[   6, 2061,  318,  262, 1110, 1909,   30,  220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
input_token_count = 
 8
tokenizer = n\ CodeGenTokenizerFast(name_or_path='./models/phi-1_5', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rs

In [13]:
print(generated_text)
print(input_token_count, output_token_count, total_token_count)

'What is the day today? 

Answer: The day today is Wednesday.

Exercise 2:

What is the difference between a week and a month?

Answer: A week is seven days long, while a month can be any number of days.

Exercise 3:

What is the difference between a year and a decade?

Answer: A year is 365 days long, while a decade is ten years long.

Exercise 4:

What is the difference between a century and a millennium?

Answer: A century is one hundred years long, while a millennium is one thousand years long.

Exercise 5:

What is the difference between a decade and a century?

Answer: A decade is ten years long, while a century is one hundred years long.



Question 4: A recipe calls for 2 cups of flour to make 12 cookies. How many cups of flour are needed to make 36 cookies?

Solution:
To find the number of cups of flour needed to make 36 cookies, we can set up a proportion using the given information.

Let x be the number of cups of flour needed to make 36 cookies.

We can set up the proportio

In [ ]:
# Все работает

## 💡 "Сейчас сделаю свой ChatGPT!"

### Типичная мысль разработчика:
> "У меня есть Transformers, есть FastAPI, есть GPU - значит, через пару часов у меня будет готовый продукт, который составит конкуренцию OpenAI!"


**Что кажется простым:**
```python
# Загрузил модель
model = AutoModelForCausalLM.from_pretrained("llama-3.2-1b")
# Обернул в FastAPI
@app.post("/generate")
def generate(text: str):
    return model.generate(text)
# Profit! 💰💰💰
```

In [ ]:
# мы использовали скрипт <fast_api_llm.py> для поднятия своего fast api сервиса с llm под капотом
# давайте протестируем этот llm сервис

In [ ]:
%run ./src/fast_api_llm.py

In [22]:
!pip install -q fastapi uvicorn transformers

!nohup uvicorn src.fast_api_llm:app --host 0.0.0.0 --port 8800 > server.log 2>&1 &

In [23]:
%%time
response = requests.post(
    'http://localhost:8800/generate',
    json={"text": 'What is the day today? '}
)

CPU times: user 16.4 ms, sys: 5.52 ms, total: 22 ms
Wall time: 7.04 s


In [24]:
response.json()

{'generated_text': 'What is the day today? \nAnswer: The day today is Wednesday.\n\n2. What is the day of the week for the following dates? \na) January 1, 2021 \nb) December 31, 2021 \nc) June 15, 2021 \nAnswer: \na) Monday \nb) Sunday \nc) Wednesday \n\n3. What is the day of the week for the following dates? \na) August 15, 2021 \nb) March 1, 2022 \nc) November 30, 2022 \nAnswer: \na) Saturday \nb) Monday \nc) Friday \n\n4. What is the day of the week for the following dates? \na) April 15, 2022 \nb) October 31, 2022 \nc) July 20, 2023 \nAnswer: \na) Sunday \nb) Wednesday \nc) Saturday \n\n5. What is the day of the week for the following dates? \na) February 28, 2023 \nb) July 4, 2023 \nc) December 25, 2023 \nAnswer: \na) Monday \nb) Thursday \nc) Sunday\n\n\n\n',
 'input_token_count': 7,
 'output_token_count': 256,
 'total_token_count': 263,
 'original_prompt': 'What is the day today? '}

In [ ]:
# Отлично!
# Какие метрики обычно снимают с API-сервисов? Давайте снимем их
# для этого воспользуемся специально написанным скриптом <api_metrics_test.py>

In [26]:
%run src/api_metrics_test.py --base-url http://localhost:8800

🧪 API Metrics Testing Tool
Target URL: http://localhost:8800
Sequential requests: 5
Concurrent requests: 10 (workers: 3) 
Stress test: 15s (workers: 3) 

✅ API health check passed!

🔄 Running 5 sequential requests...
Request 1/5: ✅ (6.04s)
Request 2/5: ✅ (6.66s)
Request 3/5: ✅ (6.06s)
Request 4/5: ✅ (6.72s)
Request 5/5: ✅ (6.02s)

🚀 Running 10 concurrent requests with 3 workers...
Request 1/10: ✅ (6.72s)
Request 2/10: ✅ (12.74s)
Request 3/10: ✅ (19.48s)
Request 4/10: ✅ (18.84s)
Request 5/10: ✅ (19.53s)
Request 6/10: ✅ (18.85s)
Request 7/10: ✅ (19.40s)
Request 8/10: ✅ (18.97s)
Request 9/10: ✅ (19.32s)
Request 10/10: ✅ (19.04s)

💪 Running stress test for 15 seconds with 3 workers...

📊 API PERFORMANCE METRICS REPORT
Timestamp: 2025-08-03 14:07:56

📈 Test Summary:
  Total Requests: 165
  Successful: 165 (100.0%)
  Failed: 0 (0.0%)

⏱️  Response Time Metrics:
  Average: 1935.46 ms
  Median: 724.47 ms
  Min: 363.48 ms
  Max: 19528.11 ms
  95th percentile: 12743.87 ms
  99th percentile: 1948

In [ ]:
# и что же получили?

### 📝 что видим

1.  **Проблема "длинного хвоста" (Tail Latency):** Существует значительный разрыв между медианным (`2.3 сек`) и средним (`3.17 сек`) временем ответа. Это указывает на наличие небольшого количества очень медленных запросов, которые сильно искажают общую картину.
    *   **Критический показатель:** Максимальное время ответа в `20.6 секунд` и 95-й перцентиль в `~8 секунд` являются недопустимо высокими для большинства интерактивных приложений. Это означает, что каждый 20-й пользователь ждет ответа 8 секунд или дольше

2.  **Низкая нагрузка теста:** Пропускная способность в `0.32 RPS` (примерно 1 запрос каждые 3 секунды) является очень низкой

## 🚨 Реальность FastAPI + Transformers: почему это не работает

### Проблемы "наивного" подхода:

**1. Управление памятью:**
- Каждый запрос создает новый KV-cache
- Модель не освобождает GPU память между запросами
- При 2-3 одновременных запросах = OUT_OF_MEMORY

**2. Batching отсутствует:**
- Каждый запрос обрабатывается отдельно
- GPU используется на 10-20% от возможностей
- Ужасная экономическая эффективность

**3. Нет очередей и приоритизации:**
- Длинные запросы блокируют короткие
- Нет возможности отменить генерацию
- Нет контроля нагрузки

In [ ]:
# как нам понять, что такого решения недостаточно? Давайте снимем дополнительные метрики

## 📈 Ключевые метрики LLM в продакшене

### Что измеряем и почему это важно?

**⚡ Time to First Token (TTFT)**
- **Что:** Время от запроса до первого токена ответа
- **Важность:** Определяет "отзывчивость" для пользователя
- **Хорошо:** < 200ms, **Плохо:** > 1000ms
- **Влияет на:** UX (User Experience) в чатах и интерактивных приложениях

**🚀 Tokens per Second (TPS)**  
- **Что:** Скорость генерации токенов после первого
- **Важность:** Определяет, как быстро "печатается" ответ
- **Хорошо:** > 50 TPS, **Плохо:** < 10 TPS
- **Влияет на:** Восприятие "умности" системы

**⏰ Inter-Token Latency (ITL)**
- **Что:** Задержка между соседними токенами
- **Важность:** Должна быть стабильной для плавного вывода
- Хорошо: < 50ms, Плохо: > 200ms
- **Влияет на:** Плавность streaming-ответов

**📊 System Throughput**
- **Что:** Общее количество токенов/запросов в секунду
- **Важность:** Определяет масштабируемость системы
- **Измеряется:** RPS (requests/sec), Tokens/sec
- **Влияет на:** Стоимость инфраструктуры и возможности роста


In [ ]:
# снимаем эти метрики с нашего сервиса
# для этого воспользуемся еще одним написанным скриптом <llm_inference_metrics.py>

In [ ]:
%%time
%run src/llm_inference_metrics.py --base-url http://localhost:8800 --endpoint /generate --api-format fastapi

🧠 LLM Inference Metrics Testing Tool
📡 Target: http://localhost:8800/generate
🔧 API Format: fastapi
📊 Total requests: ~127


🧠 LLM INFERENCE PERFORMANCE METRICS REPORT
Timestamp: 2025-07-20 17:34:33

📊 Test Summary:
  Total Requests: 127
  Successful: 127 (100.0%)
  Failed: 0

⚡ Time to First Token (TTFT):
  Average: 674.55 ms
  Median: 684.32 ms
  Min: 331.23 ms
  Max: 1098.91 ms
  95th percentile: 1069.69 ms
  Std deviation: 214.13 ms

🚀 Tokens per Second (TPS):
  Average: 7.16 tokens/sec
  Median: 5.1 tokens/sec
  Min: 3.0 tokens/sec
  Max: 24.08 tokens/sec
  95th percentile: 23.34 tokens/sec
  Std deviation: 5.27 tokens/sec

⏰ Inter-Token Latency (ITL):
  Average: 172.39 ms
  Median: 179.1 ms
  Min: 36.75 ms
  Max: 306.56 ms
  95th percentile: 297.49 ms
  Std deviation: 73.72 ms

📈 System Throughput:
  Input tokens/second: 1.41
  Output tokens/second: 6.04
  Requests/second: 0.17
  Total processing time: 729.68 seconds

🔤 Token Efficiency:
  Average input tokens: 8.08
  Average out

In [ ]:
# делаем акцент на следующие полученные метрики
#
# Average TTFT - 674.55 ms
# 95th Percentile TTFT - 1069.69 ms
# Average TPS - 7.16 tokens/sec
# Average ITL - 172.39 ms
# 95th Percentile ITL - 297.49 ms
# Output Tokens/Second - 6.04
# Requests/Second - 0.17

## 🎯 Сравнение с индустриальными стандартами

На [этом сайте](https://artificialanalysis.ai/models#latency) можно посмотреть разные бенчмарки компаний

### Что показывают наши метрики vs лидеры рынка?
Все метрики на примере модели GPT-4.1


**Наши результаты (FastAPI + Transformers):**
- Average TTFT: ~675ms ⚠️
- Average TPS: ~7 tokens/sec 🐌  
- Requests/Second: ~0.17 💀

**Лидеры индустрии:**
- TTFT: ~500ms ⚡
- TPS: ~145 tokens/sec 🚀
- Concurrent users: тысячи одновременно 🏭

### 📊 Насколько мы отстаем?

| Метрика | Наш результат | Индустрия | Отставание |
|---------|---------------|-----------|------------|
| TPS | 7 | 145 | **20x хуже** |
| TTFT | 675ms | 500ms | **1.35x хуже** |
| RPS | 0.17 | 10+ | **60x хуже** |
| Concurrent users | 1 | 1000+ | **1000x хуже** |

### 💡 Вывод:
**Сделать стартап за $40млрд с такими метриками не получится**


Большинство разработчиков думают, что развернуть LLM в проде - это просто обернуть модель в API. Но реальность гораздо сложнее!

Переход от research prototype к production system требует глубокого понимания:
- Computer Systems Architecture
- Distributed Computing Theory  
- GPU Programming Models
- Economic Optimization
- Reliability Engineering

Нужны кардинально другие подходы:
- Специализированные inference движки
- Продвинутое управление памятью  
- Continuous batching
- Оптимизированные CUDA ядра


## 🎯 Проблематика Production LLM: Четыре ключевых вызова

Просто "запустить модель на сервере" в мире LLM — это провал. В отличие от классических ML-моделей, инференс LLM создает уникальные и сложные инженерные проблемы.

### 1. **Латентность (Latency)**

Пользователи ожидают интерактивности, схожей с человеческим общением. В контексте LLM это разделяется на два показателя:

- **Time to First Token (TTFT):** Время до получения первого токена. Должно быть минимальным (< 200-500ms), чтобы пользователь видел, что система "начала отвечать".
- **Time per Output Token (TPOT):** Время генерации каждого последующего токена. Определяет "скорость печати" модели.


### 2. **Стоимость и утилизация GPU (Cost \& Utilization)**

GPU-ускорители, необходимые для работы LLM, дороги (2-15 $ в час за один GPU). Чтобы сервис был экономически выгодным, каждый доллар, вложенный в оборудование, должен приносить максимальную отдачу. **Низкая утилизация GPU — это деньги, сжигаемые впустую.**

### 3. **Масштабируемость (Scalability)**

Сервис должен одинаково хорошо работать как для одного пользователя, так и для миллионов одновременных запросов. Нагрузка на LLM-сервисы непредсказуема и может взлетать в пиковые моменты.

### 4. **Надежность (Reliability)**

Основная причина сбоев в LLM-сервисах — ошибки нехватки памяти (Out of Memory, OOM). Длинные запросы или всплеск трафика могут легко исчерпать память GPU, приводя к отказам в обслуживании. Цель — обеспечить доступность сервиса на уровне 99.9% и выше.


____
# Часть 2
Здесь мы разберем следующие темы
- 🏗️ Архитектурные паттерны инференса
- 🚄 Сравнение vLLM и TensorRT-LLM
## 🔧 Архитектурные паттерны инференса

В основе производительности LLM лежат два различных режима работы GPU: обработка промпта и генерация ответа. Понимание их различий — ключ к оптимизации.

### Фазы вычислений: Prefill vs. Decode

#### **Prefill-фаза**

Это первый этап, когда модель обрабатывает входной промпт пользователя.

* **Процесс:** Все токены промпта (`"Объясни квантовую физику простыми словами"`) обрабатываются **одновременно и параллельно**.
* **Характеристики:**
    * **Высокая загрузка вычислительных ядер GPU (Compute-bound):** На этом этапе выполняются массивные матричные умножения.
    * **Высокий параллелизм:** Идеальный сценарий для архитектуры GPU.
    * **Результат:** Расчет внутреннего состояния (KV-кэша) для всего промпта и генерация **первого** токена ответа.


#### **Decode-фаза**

Это второй этап, когда модель генерирует ответ токен за токеном.

* **Процесс:** Генерация носит авторегрессионный характер: чтобы сгенерировать токен `N`, нужно знать токен `N-1`. Это строго **последовательная** операция.
* **Характеристики:**
    * **Низкая загрузка вычислительных ядер GPU:** На каждом шаге выполняется лишь небольшая работа.
    * **Ограничение пропускной способностью памяти (Memory-bandwidth bound):** Скорость определяется тем, как быстро GPU может читать и записывать KV-кэш из своей памяти.
    * **Низкая утилизация GPU:** Большую часть времени дорогие вычислительные ядра простаивают.

> **Ключевой вывод:** Фаза **Prefill** эффективно использует GPU, в то время как фаза **Decode** — крайне неэффективно. Именно оптимизация медленной Decode-фазы приносит наибольший выигрыш в производительности.

In [ ]:
# подробнее - некоторые важные особенности
# https://www.bentoml.com/llm/inference-optimization/prefill-decode-disaggregation

In [ ]:
# как решить эту проблему?

### KV Cache: Память, которая всё меняет (и съедает)

**Что это?**
KV-кэш — это "краткосрочная память" модели для текущего запроса. В механизме внимания (attention) модели необходимо иметь доступ к информации о всех предыдущих токенах в последовательности. KV-кэш хранит предварительно вычисленные векторы ключей (Keys) и значений (Values) для каждого токена. Это избавляет от необходимости пересчитывать их заново при генерации каждого нового токена.

**Проблема:**


Размер KV-кэша растет линейно с длиной последовательности и является главным потребителем видеопамяти (VRAM).

```python
# Формула для расчета размера KV-кэша на один запрос
# (для float16, где каждый параметр занимает 2 байта)

kv_cache_size_bytes = (
      num_layers * 2  # K и V
    * hidden_size
    * sequence_length
    * batch_size
    * 2  # байт на элемент (float16)
)

# Пример для Llama-3-8B с длиной контекста 4096 токенов:
# 32 layers * 2 * 4096 hidden_size * 4096 tokens * 1 batch * 2 bytes = ~2.1 GB
```

**Почему это критично:**

1. **Ограничивает Batch Size:** Чем больше памяти съедает один запрос, тем меньше одновременных запросов может обработать GPU.
2. **Ограничивает максимальный контекст:** Доступная VRAM напрямую определяет, какой длины промпты и ответы вы можете поддерживать.
3. **Главная причина OOM-ошибок:** Большой входящий промпт или пик трафика мгновенно исчерпывают память.




In [ ]:
# а еще проблемы будут?


### Проблема: Статический батчинг (Static Batching)

Традиционный подход к батчингу, унаследованный из обучения моделей.

* **Как работает:** Несколько запросов объединяются в один батч. Модель обрабатывает батч целиком.
* **Главный недостаток:** Все последовательности в батче должны иметь одинаковую длину. Короткие запросы дополняются "пустыми" токенами (padding) до длины самого длинного запроса.
* **Результат:** GPU тратит драгоценные циклы на обработку бесполезных padding-токенов. Более того, весь батч не завершится, пока не будет сгенерирован ответ на самый длинный запрос.

> **Аналогия:** Автобус, который ждет, пока *абсолютно все* его пассажиры доберутся до своих конечных остановок, прежде чем сможет взять новых, даже если половина мест освободилась в самом начале маршрута.

In [ ]:
# как решить эту проблему?

### Решение: Непрерывный батчинг (Continuous Batching)

Этот подход, популяризированный фреймворком **vLLM**, вносит кардинальные изменения

* **Как работает:** Вместо батчинга на уровне всего батча, управление происходит на уровне отдельных запросов.

1. Как только генерация для одного запроса в батче завершается, он немедленно удаляется.
2. В освободившееся в памяти GPU место тут же помещается новый ожидающий запрос.
3. Таким образом, батч постоянно "пересобирается", а GPU никогда не простаивает и не тратит время на padding.
* **Ключевой механизм:** Для реализации этого используется техника **PagedAttention**, которая управляет KV-кэшем аналогично тому, как операционная система управляет виртуальной памятью — делит память на "страницы" и динамически выделяет их запросам. Это решает проблему фрагментации памяти.

> **Аналогия:** Ресторан с эффективным хостес. Как только столик освобождается, за него сразу сажают следующую компанию из очереди. Ни одно место не простаивает.

**Результат:** **Увеличение пропускной способности (throughput) в 2–20 раз** по сравнению со статическим батчингом. Это одна из самых важных оптимизаций для современных LLM-сервисов.



In [ ]:
# не самим же все это писать?

## Обзор ключевых фреймворков: vLLM и TensorRT-LLM

Среди множества движков для инференса выделяются два основных решения для высокопроизводительных вычислений на GPU, каждое со своей специализацией.

### **vLLM: Фокус на управлении памятью и пропускной способности**

**vLLM** — это [библиотека с открытым исходным](https://github.com/vllm-project/vllm) кодом от UC Berkeley, которая предложила подход, значительно повышающий эффективность инференса.

*   **Основной принцип:** Ключевая проблема при обслуживании LLM-запросов заключается в неэффективном управлении памятью, выделяемой под KV-кэш. Оптимизация этого аспекта напрямую ведет к росту пропускной способности.
*   **Ключевая технология: PagedAttention**
    *   По аналогии с системами виртуальной памяти в ОС, PagedAttention организует KV-кэш в виде не-непрерывных блоков (страниц) памяти.
    *   Это устраняет проблему внешней фрагментации памяти, из-за которой значительная часть VRAM могла оставаться незадействованной.
    *   В результате достигается утилизация памяти с эффективностью выше 96%.
*   **Основная функциональность: Continuous Batching**
    *   Эффективное управление памятью через PagedAttention является основой для реализации непрерывного батчинга.
    *   Система динамически управляет составом батча, добавляя новые запросы по мере освобождения ресурсов, что обеспечивает постоянную и высокую загрузку GPU

[подробнее тут](https://www.ubicloud.com/blog/life-of-an-inference-request-vllm-v1)

### **TensorRT-LLM: Фокус на компиляции и низкоуровневых оптимизациях**

**TensorRT-LLM** — это [библиотека от NVIDIA](https://github.com/NVIDIA/TensorRT-LLM), предназначенная для достижения пиковой производительности LLM на графических процессорах компании.

*   **Основной принцип:** Обеспечение максимальной производительности за счет компиляции модели в высокооптимизированный исполняемый код, адаптированный под конкретную архитектуру GPU (например, Hopper, Ampere) и параметры задачи.
*   **Ключевая технология: Компиляция и CUDA-оптимизации**
    *   **Kernel Fusion (Слияние ядер):** Несколько последовательных операций CUDA объединяются в одно оптимизированное ядро, что значительно сокращает накладные расходы на вызовы и обращения к глобальной памяти GPU.
    *   **Графовые оптимизации:** Анализ и преобразование вычислительного графа модели для более эффективного исполнения.
    *   **Поддержка числовых форматов:** Использование форматов пониженной точности, таких как FP8 на новейших архитектурах, для ускорения вычислений и снижения требований к памяти.
*   **Основная функциональность:** TensorRT-LLM интегрирует собственные реализации PagedAttention и In-flight Batching, сочетая их с глубокими аппаратными оптимизациями

In [ ]:
# значит будем сравнивать vLLM и TensorRT - с использованием контейнера triton от nvidia

In [ ]:
# начнем с vLLM

In [ ]:
# у нас есть запущенный контейнер, подробная инструкция есть в папке triton_instructions

In [ ]:
# пробуем постучаться в контейнер

In [ ]:
%%time
payload = {
    "text_input": 'What is the day today? ',
    "stream": False,
    "max_tokens": 15,
    "temperature": 0.1
}

response = requests.post(
    "http://localhost:8000/v2/models/llama3_2_1b_local/generate",
    json=payload
)

CPU times: total: 0 ns
Wall time: 385 ms


In [ ]:
response.json()

{'model_name': 'llama3_2_1b_local',
 'model_version': '1',
 'text_output': 'What is the day today?  I am trying to figure out what day of the week it is today.'}

In [ ]:
# работает!

In [ ]:
# собираем метрики, как для нашего fast api сервиса

In [ ]:
%%time
%run src/llm_inference_metrics.py --base-url http://localhost:8000 --endpoint /v2/models/llama3_2_1b_local/generate --api-format triton

🧠 LLM Inference Metrics Testing Tool
📡 Target: http://localhost:8000/v2/models/llama3_2_1b_local/generate
🔧 API Format: triton
📊 Total requests: ~127


🧠 LLM INFERENCE PERFORMANCE METRICS REPORT
Timestamp: 2025-07-20 17:43:24

📊 Test Summary:
  Total Requests: 127
  Successful: 127 (100.0%)
  Failed: 0

⚡ Time to First Token (TTFT):
  Average: 120.73 ms
  Median: 115.78 ms
  Min: 34.87 ms
  Max: 210.93 ms
  95th percentile: 208.38 ms
  Std deviation: 29.29 ms

🚀 Tokens per Second (TPS):
  Average: 81.28 tokens/sec
  Median: 81.6 tokens/sec
  Min: 48.66 tokens/sec
  Max: 102.39 tokens/sec
  95th percentile: 95.41 tokens/sec
  Std deviation: 8.13 tokens/sec

⏰ Inter-Token Latency (ITL):
  Average: 9.1 ms
  Median: 8.93 ms
  Min: 7.19 ms
  Max: 15.04 ms
  95th percentile: 10.55 ms
  Std deviation: 0.98 ms

📈 System Throughput:
  Input tokens/second: 18.26
  Output tokens/second: 82.44
  Requests/second: 2.32
  Total processing time: 54.66 seconds

🔤 Token Efficiency:
  Average input token

In [ ]:
# сравним

## Сравнение показателей

| Метрика                  | FastAPI                 | vLLM                  | Улучшение           |
|--------------------------|------------------------|-----------------------|---------------------|
| Average TTFT             | 674.55 ms              | 120.73 ms             | ~5.6x быстрее       |
| 95th Percentile TTFT     | 1069.69 ms             | 208.38 ms             | ~5.1x быстрее       |
| Average TPS              | 7.16 tokens/sec        | 81.28 tokens/sec      | ~11.4x выше         |
| Average ITL              | 172.39 ms              | 9.1 ms                | ~19x быстрее        |
| 95th Percentile ITL      | 297.49 ms              | 10.55 ms              | ~28x быстрее        |
| Output Tokens/Second     | 6.04                   | 82.44                 | ~13.6x выше         |
| Requests/Second          | 0.17                   | 2.32                  | ~13.6x выше         |

- **Time to First Token (TTFT):** vLLM значительно уменьшает как средний, так и 95-й процентиль задержки первого токена.
- **Token Per Second (TPS):** vLLM обеспечивает более чем 10-кратное увеличение скорости генерации.
- **Output Tokens/Second и Requests/Second:** Рост пропускной способности при применении vLLM наблюдается почти на порядок.

In [ ]:
# теперь проверяем TensorRT-LLM

## Зачем нужна компиляция модели с помощью TensorRT-LLM?

**Компиляция** с помощью TensorRT-LLM — это процесс преобразования стандартной, "универсальной" языковой модели (например, из Hugging Face) в сверхэффективный, специализированный **"движок" (engine)**. Этот движок создан для максимально быстрой работы на **конкретной модели GPU NVIDIA**

Если вы планируете использовать LLM не для экспериментов, а в реальном продукте, этот шаг становится критически важным.

### Ключевые цели оптимизации

Компиляция решает три главные задачи, без которых развертывание LLM становится неэффективным и дорогим

1.  **⬇️ Низкая задержка (Low Latency):** Значительно сокращается время генерации ответа. Это то, что напрямую влияет на пользовательский опыт.
2.  **⬆️ Высокая пропускная способность (High Throughput):** Та же видеокарта может обслуживать гораздо больше запросов одновременно. Это напрямую снижает затраты на инфраструктуру
3.  **🧠 Эффективное использование памяти:** Оптимизированная модель потребляет меньше видеопамяти (VRAM), что позволяет запускать более крупные модели на том же оборудовании

Согласно тестам NVIDIA, прирост в скорости может достигать **8 раз** по сравнению с неоптимизированными версиями

### Что происходит "под капотом" во время компиляции?

Когда вы запускаете команду `trtllm-build`, TensorRT-LLM выполняет глубокую оптимизацию модели под ваше "железо". Основные техники включают:

*   **Слияние слоев (Layer Fusion):** Вместо выполнения множества мелких операций (умножение матриц, сложение, функция активации) последовательно, TensorRT объединяет их в одно большое, оптимизированное ядро CUDA. Это резко сокращает обращения к памяти GPU.
*   **Квантование (Quantization):** Веса и вычисления модели переводятся в форматы с пониженной точностью (например, **FP16**, **INT8**). Эти форматы обрабатываются значительно быстрее на современных Tensor Cores в GPU NVIDIA и требуют меньше памяти
*   **Автоматический подбор ядер (Kernel Auto-Tuning):** TensorRT тестирует различные алгоритмы для выполнения операций на вашем GPU и выбирает самый быстрый из них
*   **Продвинутое управление памятью:** Внедряются такие техники, как **Paged Attention** и **In-flight Batching**, которые оптимизируют использование K/V кэша, позволяя эффективно обрабатывать множество запросов разной длины одновременно

### Практические выводы и компромиссы

| Характеристика | Стандартная модель (Hugging Face) | Скомпилированная модель (TensorRT-LLM) |
| :--- | :--- | :--- |
| **Скорость** | Стандартная | Максимально возможная для данного GPU |
| **Пропускная способность** | Базовая | Значительно выше |
| **Потребление VRAM** | Высокое | Сниженное |
| **Гибкость** | **Универсальность:** работает на любом GPU | **Специализация:** привязана к конкретной модели GPU |

> **❗Важно помнить:** Скомпилированный движок **не является переносимым**. Если вы захотите запустить модель на другой модели GPU или измените параметры самой модели (например, точность), вам придется **выполнить компиляцию заново**. Вы компилируете модель один раз для конкретной связки "модель + параметры + GPU" и затем многократно используете полученный движок

In [ ]:
# имеем на руках развернутый контейнер - пробуем в него постучаться

In [ ]:
%%time

url = "http://localhost:8000/v2/models/ensemble/infer"
data = {
    "inputs": [
        {
            "name": "text_input",
            "shape": [1, 1],
            "datatype": "BYTES",
            "data": ["What is deep learning?"]
        },
        {
            "name": "max_tokens",
            "shape": [1, 1],
            "datatype": "INT32",
            "data": [80]
        }
    ]
}

response = requests.post(url, json=data)
print(response.json())

{'model_name': 'ensemble', 'model_version': '1', 'parameters': {'sequence_id': 0, 'sequence_start': False, 'sequence_end': False}, 'outputs': [{'name': 'text_output', 'datatype': 'BYTES', 'shape': [1], 'data': [' Deep learning is a type of artificial intelligence (AI) that uses algorithms and statistical models to analyze and interpret complex data. It is a subset of machine learning, which is a broader field of study that involves the use of algorithms and statistical models to enable machines to perform tasks that typically require human intelligence.\n\nDeep learning is particularly useful for tasks that involve:\n\n1. Image and video analysis\n2. Natural']}]}
CPU times: total: 0 ns
Wall time: 1.07 s


In [ ]:
# собираем метрики

In [ ]:
%%time
%run src/llm_inference_metrics.py --base-url http://localhost:8000 --endpoint /v2/models/ensemble/infer --api-format tensorrt

🧠 LLM Inference Metrics Testing Tool
📡 Target: http://localhost:8000/v2/models/ensemble/infer
🔧 API Format: tensorrt
📊 Total requests: ~127


🧠 LLM INFERENCE PERFORMANCE METRICS REPORT
Timestamp: 2025-07-20 17:39:56

📊 Test Summary:
  Total Requests: 127
  Successful: 127 (100.0%)
  Failed: 0

⚡ Time to First Token (TTFT):
  Average: 121.06 ms
  Median: 106.74 ms
  Min: 36.74 ms
  Max: 205.88 ms
  95th percentile: 203.91 ms
  Std deviation: 35.32 ms

🚀 Tokens per Second (TPS):
  Average: 82.01 tokens/sec
  Median: 87.67 tokens/sec
  Min: 42.41 tokens/sec
  Max: 105.4 tokens/sec
  95th percentile: 99.89 tokens/sec
  Std deviation: 16.59 tokens/sec

⏰ Inter-Token Latency (ITL):
  Average: 9.52 ms
  Median: 8.6 ms
  Min: 6.91 ms
  Max: 18.19 ms
  95th percentile: 15.45 ms
  Std deviation: 2.72 ms

📈 System Throughput:
  Input tokens/second: 17.92
  Output tokens/second: 80.05
  Requests/second: 2.28
  Total processing time: 55.7 seconds

🔤 Token Efficiency:
  Average input tokens: 7.86
  

In [ ]:
# сравним

## сравнение

| Метрика                  | FastAPI (Базовый)      | vLLM (Оптимизированный) | TensorRT-LLM (Оптимизированный) |
|--------------------------|------------------------|-------------------------|---------------------------------|
| Average TTFT             | 674.55 ms              | 120.73 ms               | 121.06 ms                       |
| 95th Percentile TTFT     | 1069.69 ms             | 208.38 ms               | 203.91 ms                       |
| Average TPS              | 7.16 tokens/sec        | 81.28 tokens/sec        | 82.01 tokens/sec                |
| Average ITL              | 172.39 ms              | 9.1 ms                  | 9.52 ms                         |
| Requests/Second          | 0.17                   | 2.32                    | 2.28                            |

Данные демонстрируют, что как vLLM, так и TensorRT-LLM показывают схожие, кардинальные улучшения по сравнению с базовым подходом:

-   **Латентность (TTFT, ITL):** Снижение в 5-20 раз.
-   **Пропускная способность (TPS, Requests/Second):** Увеличение более чем в 10 раз.

Это подтверждает, что использование специализированных движков инференса является ключевым фактором для достижения высокой производительности

# Где и кто применяет TensorRT-LLM и vLLM

| Компания / организация | Продукт или сервис | Движок инференса | Как используется | Ключевые эффекты |
|------------------------|--------------------|------------------|------------------|------------------|
| **[Amazon](https://pytorch.org/blog/vllm-joins-pytorch/)** | Shopping-ассистент Rufus | vLLM (на AWS Inferentia / Trainium) | Обслуживает миллионы запросов в мобильном приложении, выводя первый токен < 1 с даже в периоды Prime Day | 3 млн токенов/мин при P99 TTFT < 1 с |
| **[Microsoft](https://ppc.land/bing-optimizes-search-speed-with-tensorrt-llm-cutting-model-latency-by-36-percent/)** | Bing Search / Deep Search | TensorRT-LLM | Оптимизация SLM-моделей, уменьшение 95-й процентильной задержки с 4.76 с до 3.03 с и рост пропускной способности на 57% | Снижение затрат и ускорение поисковой выдачи |
| **IBM** | Watsonx-платформа и внутренние AI-службы | vLLM | Производственный инференс, активное участие в разработке | Повышенная пропускная способность при снижении VRAM-затрат |
| **[LinkedIn](https://blog.vllm.ai/2025/01/10/vllm-2024-wrapped-2025-vision.html)** | AI-функции (рекомендации, сообщения) | vLLM | Генеративные подсказки и контент-ассистенты | Масштабирование на много миллионов пользователей |
| **Snowflake, Red Hat, Anyscale и др.** | Облачные и on-prem AI-сервисы | vLLM | Интеграция в хостинг-платформы для LLM-моделей | Упрощённый OpenAI-совместимый API, экономия GPU-часов |
| **DeepSeek AI** | DeepSeek-V3 / R1 | TensorRT-LLM | Релиз INT4/BF16 вариантов и официальные примеры | До 8× более быстрый вывод на H100 |
| **[NVIDIA Jetson AI Lab](https://www.jetson-ai-lab.com/tensorrt_llm.html)** | Jetson AGX Orin Edge-Inference | TensorRT-LLM | Сборки под ARM-GPU, INT4 Llama-7B | Работа LLM-ов на встраиваемых устройствах |

In [ ]:
# уже стартап? Уже можем обслуживать highload? Да, но нужно знать, а как расти и масштабироваться

## Конфигурации для масштабирования и параллелизма с TensorRT-LLM и Triton

### Стратегии распределения нагрузки

#### **1. Tensor Parallelism (TP) — Разделение весов модели**

Tensor Parallelism распределяет веса модели между несколькими GPU для моделей, которые не помещаются в память одной карты

**Пример конфигурации для LLaMA 70B на 4 GPU:**

```bash
# Конвертация checkpoint с TP=4
python3 convert_checkpoint.py --model_dir ./llama-70b-hf/ \
                             --output_dir ./llama_checkpoint_4gpu_tp4 \
                             --dtype float16 \
                             --tp_size 4

# Сборка движка
trtllm-build --checkpoint_dir ./llama_checkpoint_4gpu_tp4 \
            --output_dir ./engines/llama-70b/fp16/4-gpu/ \
            --gemm_plugin float16 \
            --gpt_attention_plugin float16 \
            --paged_kv_cache enable
```

**Переменные окружения для Triton:**
```bash
export WORLD_SIZE=4
export TP_SIZE=4
export PP_SIZE=1
export GPU_DEVICE_IDS="0,1,2,3"
export INSTANCE_COUNT=1
```

#### **2. Pipeline Parallelism (PP) — Разделение слоев модели**

Pipeline Parallelism разделяет слои модели по разным GPU/нодам, создавая конвейер обработки

**Пример для LLaMA 70B с TP=4 и PP=2 (8 GPU):**

```bash
# Конвертация с Pipeline Parallelism
python3 convert_checkpoint.py --model_dir ./llama-70b-hf/ \
                             --output_dir ./llama_checkpoint_8gpu_tp4_pp2 \
                             --dtype float16 \
                             --tp_size 4 \
                             --pp_size 2

trtllm-build --checkpoint_dir ./llama_checkpoint_8gpu_tp4_pp2 \
            --output_dir ./engines/llama-70b/8-gpu/ \
            --gemm_plugin auto
```

**Переменные для multi-node deployment:**
```bash
export WORLD_SIZE=8
export TP_SIZE=4
export PP_SIZE=2
export GPUS_PER_NODE=4
export TRITON_BACKEND="tensorrtllm"
```

#### **3. Data Parallelism — Множественные копии модели**

Запуск нескольких независимых копий модели для увеличения пропускной способности

**Конфигурация Triton для multiple instances:**
```bash
export ENGINE_DIR="/workspace/trt_engines"
export TOKENIZER_DIR="/workspace/local_models/llama-3.2-7B-Instruct"
export INSTANCE_COUNT=4  # 4 копии модели
export MAX_BATCH_SIZE=8
export GPU_DEVICE_IDS="0,1,2,3"  # Разные GPU для каждой копии
```

### **Продвинутая оптимизация KV Cache**

#### **1. Базовая конфигурация Paged KV Cache**

```bash
export MAX_TOKENS_IN_PAGED_KV_CACHE=8192
export MAX_ATTENTION_WINDOW_SIZE=4096
export KV_CACHE_FREE_GPU_MEM_FRACTION=0.7
export ENABLE_KV_CACHE_REUSE=True
export PAGED_KV_CACHE=enable
```

#### **2. KV Cache Offloading на CPU память**

Для поддержки более длинных контекстов

**TensorRT-LLM конфигурация:**
```bash
# При сборке движка
trtllm-build --checkpoint_dir ./checkpoint \
            --output_dir ./engines \
            --paged_kv_cache enable \
            --kv_cache_host_memory_bytes 32000000000  # 32GB в host памяти
```

**Triton переменные для host offloading:**
```bash
export ENABLE_KV_CACHE_HOST_OFFLOAD=1
export KV_CACHE_HOST_MEM_FRACTION=0.2  # 20% host памяти для KV кэша
export KV_CACHE_FREE_GPU_MEM_FRACTION=0.5  # Остальное в GPU
```

**Для GH200/GB200 с unified memory:**
```bash
export NIM_ENABLE_KV_CACHE_HOST_OFFLOAD=1
export NIM_KV_CACHE_HOST_MEM_FRACTION=0.15
```

### Комплексные конфигурации для различных сценариев

#### **Сценарий 1: Высокая пропускная способность (Multi-Instance)**

**config.pbtxt для ensemble модели:**
```protobuf
name: "ensemble_model"
platform: "ensemble"
max_batch_size: 256

instance_group [
  {
    count: 4
    kind: KIND_GPU
    gpus: [ 0, 1, 2, 3 ]
  }
]

ensemble_scheduling {
  step [
    {
      model_name: "preprocessing"
      model_version: -1
      input_map {
        key: "INPUT_TEXT"
        value: "input_text"
      }
      output_map {
        key: "INPUT_IDS"
        value: "preprocessed_input"
      }
    },
    {
      model_name: "tensorrt_llm"
      model_version: -1
      input_map {
        key: "input_ids"
        value: "preprocessed_input"
      }
      output_map {
        key: "output_ids"
        value: "llm_output"
      }
    }
  ]
}
```

#### **Сценарий 2: Низкая задержка с оптимизацией**

```bash
# Переменные для минимальной задержки
export DECOUPLED_MODE=true  # Стриминг ответа
export MAX_QUEUE_DELAY_MICROSECONDS=0
export BATCHING_STRATEGY="inflight_fused_batching"
export MAX_BATCH_SIZE=4  # Меньший батч для латентности
export ENABLE_KV_CACHE_REUSE=True
export MAX_NUM_TOKENS=2048
```

#### **Сценарий 3: Экстремальное масштабирование (Multi-node)**

**Для распределенного развертывания:**
```bash
# Node 1
export WORLD_SIZE=16
export TP_SIZE=8
export PP_SIZE=2
export GPUS_PER_NODE=8
export NODE_RANK=0
export MASTER_ADDR="node1_ip"
export MASTER_PORT="29500"

# Node 2
export WORLD_SIZE=16
export TP_SIZE=8  
export PP_SIZE=2
export GPUS_PER_NODE=8
export NODE_RANK=1
export MASTER_ADDR="node1_ip"
export MASTER_PORT="29500"
```

**Запуск Triton с MPI:**
```bash
python3 /app/scripts/launch_triton_server.py \
    --world_size=16 \
    --model_repo=/models
```

### **Конфигурации для квантования**

#### **FP8 Quantization (Hopper архитектура)**

```bash
# При конвертации checkpoint
python3 convert_checkpoint.py --model_dir ./model \
                             --output_dir ./checkpoint \
                             --dtype float16 \
                             --quantization fp8

# При сборке движка
trtllm-build --checkpoint_dir ./checkpoint \
            --output_dir ./engines \
            --gemm_plugin fp8 \
            --gpt_attention_plugin fp8
```

#### **INT8 SmoothQuant**

```bash
# Переменные для INT8
export LOGITS_DATATYPE="TYPE_FP32"
export ENCODER_INPUT_FEATURES_DATA_TYPE="TYPE_FP16"
export QUANTIZATION_MODE="int8_smoothquant"
```

### **Мониторинг и отладка**

```bash
# Переменные для подробных логов
export TRITON_LOG_VERBOSE=2
export CUDA_VISIBLE_DEVICES="0,1,2,3"
export NCCL_DEBUG=INFO
export NCCL_IB_DISABLE=0  # Для InfiniBand

# Метрики производительности
export TRITON_ENABLE_METRICS=true
export TRITON_METRICS_PORT=8002
```

### **Практические рекомендации**

1. **Выбор стратегии параллелизма:**
   - **Single-Node Multi-GPU**: Используйте Tensor Parallelism
   - **Multi-Node**: Комбинируйте TP + PP (TP = GPUs per node, PP = количество nodes)
   - **Высокая нагрузка**: Data Parallelism с множественными экземплярами

2. **Оптимизация памяти:**
   - Начните с `KV_CACHE_FREE_GPU_MEM_FRACTION=0.9`
   - При нехватке памяти включите host offloading
   - Используйте квантование для больших моделей

3. **Настройка батчинга:**
   - `MAX_BATCH_SIZE` = от 1 (латентность) до 256+ (пропускная способность)
   - `MAX_NUM_TOKENS` должно соответствовать вашим typical request размерам
   - `inflight_fused_batching` для лучшей утилизации

____
# Часть 3
Ракроем следующие темы
- 💰 Экономика self-hosting vs API

In [ ]:
# у нас много GPU, у нас небольшой стартап, а это вообще выгодно и эффективно?
# не надо ли было платить за LLM API?
# для чего вообще начинать разворачивать свою систему?

### Вводные для расчетов
- **API стоимость:** 200 руб за 1 млн токенов  
- **Self-hosting:** фиксированная стоимость 40,000 руб/месяц независимо от объема

| Объем (млн токенов) | API стоимость | Self-hosting | Экономия |
|---------------------|---------------|--------------|----------|
| 1                   | 200 руб       | 40,000 руб   | -39,800 руб |
| 50                  | 10,000 руб    | 40,000 руб   | -30,000 руб |
| 100                 | 20,000 руб    | 40,000 руб   | -20,000 руб |
| 200                 | 40,000 руб    | 40,000 руб   | 0 руб |
| 500                 | 100,000 руб   | 40,000 руб   | +60,000 руб |

### Альтернативные конфигурации GPU

**Сравнение различных GPU-решений:**
- **RTX 4090 24GB:** 40,000 руб/мес → точка окупаемости 200 млн токенов
- **A100 40GB:** 105,372 руб/мес → точка окупаемости 526.9 млн токенов

### 🎯 Когда выбирать self-hosting?

**Self-hosting выгоден когда:**
- ✅ **Высокий объем**: > 200M токенов/день
- ✅ **Предсказуемая нагрузка**: Стабильный трафик
- ✅ **Data privacy**: Критична конфиденциальность
- ✅ **Кастомизация**: Нужен fine-tuning, специфические optimizations
- ✅ **Latency requirements**: < 100ms TTFT

**API выгоден когда:**
- ✅ **Низкий объем**: < 10M токенов/день
- ✅ **Нерегулярная нагрузка**: Spike в использовании
- ✅ **Быстрый MVP**: Нужно быстро запустить продукт
- ✅ **Ограниченные ресурсы**: Нет команды для поддержки инфраструктуры

In [ ]:
print('well done')

well done
